# Own Implementation of Naive Bayes

getting necessary libraries (Run this cell)

In [1]:
import matplotlib.pyplot as plt
import copy
import numpy as np
import pandas as pd
from nltk.tokenize import TweetTokenizer
import string
import pickle
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix

getting stop words and punctuations (Run this cell)

In [29]:
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))
temp = TweetTokenizer()
punctuation = [' ', '!', '(', ')', '()', '-', '[', ']', '[]', '{}', '{', '}', ';', ':', '\', ', '<', '>', '.', '/', '?', '@', '#', '$', '%', '^', '&', '*', '_', '~']

making 2 list :
1st for training data files
2nd for testing data files
(Run this cell)

Getting training & testing results and training & testing documents

In [21]:
import os
train_files = []
test_files = []
train_y = []
test_y = []
curDir = os.getcwd()
print(curDir)
direct = os.listdir(curDir)
a = direct.index('20_newsgroups')
b = direct.index('mini_newsgroups')
#remove this .py file and checkpoints file
a = direct[a]
b = direct[b]
direct = [a, b]
print(direct)
i = -1
folderList = 0
#go through each folder
for fold in direct:
    i += 1
    path = fold
    folderlist = os.listdir(path)
    if ".DS_Store" in folderlist:
        folderlist.remove('.DS_Store')
    if ".ipynb" in folderlist:
        folderlist.remove('.ipynb')
    folderList = folderlist
    #go through the folder list having different folders.
    for folder in folderlist:
        filelist = os.listdir(path + '/' + folder)
        #go through different files of single folder.
        for file in filelist:
            #update path of each file and class of each file.
            if(fold == '20_newsgroups'):
                train_y.append(folder)
                train_files.append((path + '/' + folder + '/' + file, i))
            else:
                test_y.append(folder)
                test_files.append((path + '/' + folder + '/' + file, i))


C:\Users\Vedant Mehra\Text Classification Project
['20_newsgroups', 'mini_newsgroups']


# Using saved data to save time
loading previously computed data into data frame and dictionary respectively (Run this cell)

In [22]:
pickle_in = open("test_data_frame.txt", 'rb')
test_data_frame = pickle.load(pickle_in)
pickle_in = open("dictionary.txt", 'rb')
dictionary1 = pickle.load(pickle_in)
pickle_in = open("dataframe.txt", 'rb')
data_frame = pickle.load(pickle_in)
pickle_in = open("count_class_dictn.txt", 'rb')
count_class_dictionary = pickle.load(pickle_in)
count_classes_dictionary = count_class_dictionary 
pickle_in = open("count_class_dictn.txt", 'rb')
count_class_dictionary = pickle.load(pickle_in)
count_classes_dictn = count_class_dictionary
pickle_in = open("test_result.txt", 'rb')
test_result = pickle.load(pickle_in)


dictionary = dictionary1
direct = os.listdir(curDir)
unique_train_y = set(train_y)
train_y = pd.DataFrame(train_y)
df = data_frame
df_train = copy.deepcopy(df)
df['result'] = train_y
print(len(train_y))


19997


# can skip to run code to save time

creating dictionary to create words as features

In [6]:
dictionary = {}
i = 0
#go through each file
for file in train_files:
    current_file = open(file[0], 'r')
    x = current_file.readlines()
    #go through each line
    for line in x:
        words = temp.tokenize(line)
        #go through each word and build vocabulary with word count
        for singleWord in words:
            if(singleWord in dictionary):
                dictionary[singleWord] += 1
            else:
                dictionary[singleWord] = 1

Getting into form of x-y frame.
Building dictionary of word as key and it's frequency as value

deleting stop words and punctuations

In [8]:
punctuation = string.punctuation
for word in punctuation:
    if word in dictionary:
        del dictionary[word]
for word in stopWords:
    if word in dictionary:
        del dictionary[word]

Removing words with very less count and significance

In [6]:
new_dict = copy.deepcopy(dictionary)
for word in new_dict.keys():
    if(new_dict[word] < 300):
        del dictionary[word]
print(len((dictionary.keys())))

Creating a data frame initialised before using to fit our data

In [8]:
vec = np.arange(0, len(dictionary.keys()) * len(train_files)) * 0 + 1
df = pd.DataFrame(vec.reshape(-1, len(dictionary.keys())))
df.columns = dictionary.keys()
print(df.shape)

(19997, 2063)


Putting values for words in our Data Frame  making the data in x - y form

(Can skip running to save time)

In [15]:
i = 0
#go through each file
for file in train_files:
    current_file = open(file[0], 'r')
    x = current_file.readlines()
    #go through each line
    for line in x:
        words = temp.tokenize(line)
        #go through each word
        #remove punctuations, stopWords and digits.
        for word in punctuation:
            if word in words:
                words.remove(word)
        for word in stopWords:
            if word in dictionary:
                words.remove(word)
        nw = copy.deepcopy(words)
        for word in nw:
            if word.isdigit():
                words.remove(word)
        
        #go through each word and build vocabulary with word count
        for singleWord in words:
            if(singleWord in dictionary):
                if(type(singleWord) is int):
                    continue
                df[singleWord][i] += 1
    i += 1

# Fit Function
This function creates a dictionary of dictionaries that has count of words in each class and total words of a class.
Can skip to save time.

In [ ]:
i = 0
#go through each file

count_classes_dictn = {}
for file in train_files:
    current_class_dictn = {}
    current_file = open(file[0], 'r')
    x = current_file.readlines()
    #go through each line
    
    for line in x:
        words = temp.tokenize(line)
        #go through each word
        
        for singleWord in words:
            if(singleWord in current_class_dictn):
                current_class_dictn[singleWord] += 1
            else:
                current_class_dictn[singleWord] = 1
                
    #for each file update the data frame.
    new_dict = copy.deepcopy(current_class_dictn)
    #remove punctuations, stopWords and other unecessary words
    for word in punctuation:
        if word in new_dict:
            del current_class_dictn[word]
            
    new_dict = copy.deepcopy(current_class_dictn)
    for word in stopWords:
        if word in new_dict:
            del current_class_dictn[word]
            
    new_dict = copy.deepcopy(current_class_dictn)
    for key in new_dict:
        if key not in dictionary:
            del current_class_dictn[key]
            
    new_dict = copy.deepcopy(current_class_dictn)
    for key in dictionary:
        if key not in new_dict:
            current_class_dictn[key] = 1
    
    #updating dictionaries for different classes
    if train_y[0][i] not in count_classes_dictn.keys():
        count_classes_dictn[train_y[0][i]] = {}
    for word in current_class_dictn:
        if word in count_classes_dictn[train_y[0][i]]:
            count_classes_dictn[train_y[0][i]][word] += current_class_dictn[word]
        else:
            count_classes_dictn[train_y[0][i]][word] = current_class_dictn[word]
            
    #maintaining total count of words
    if 'total_count_class_words' in count_classes_dictn[train_y[0][i]]:
        count_classes_dictn[train_y[0][i]]['total_count_class_words'] += sum(current_class_dictn.values())
    else:
        count_classes_dictn[train_y[0][i]]['total_count_class_words'] = sum(current_class_dictn.values())
    i += 1

In [ ]:
pickle_out = open('count_class_dictn.txt', 'wb')
pickle.dump(count_classes_dictn, pickle_out)
pickle_out.close()

Using the dictionary to predict the testing files class

In [24]:
#go through each file
i = 0
test_result = []
for file in test_files:
    current_file = open(file[0], 'r')
    x = current_file.readlines()
    current_dict_test = {}
    #go through each line
    
    for line in x:
        words = temp.tokenize(line)
        #go through each word
        
        for singleWord in words:
            if(singleWord in current_dict_test):
                current_dict_test[singleWord] += 1
            else:
                current_dict_test[singleWord] = 1
    
    #deleting stop words and punctuation
    new_dict = copy.deepcopy(current_dict_test)
    for word in punctuation:
        if word in new_dict.keys():
            del current_dict_test[word]
            
    new_dict = copy.deepcopy(current_dict_test)
    for word in stopWords:
        if word in new_dict.keys():
            del current_dict_test[word]
            
    new_dict = copy.deepcopy(current_dict_test)
    for word in new_dict.keys():
        if word not in dictionary.keys():
            del current_dict_test[word]
            
    #number of words in a class c with word w
    total_class_count = len(train_y)
    max_proba, result_class = 0, 0
    flag = True
    
    for class1 in unique_train_y:
        df1 = df[df['result'] == class1]
        count_of_this_class = len(df1)
        prob_this_class = count_of_this_class / total_class_count
        #getting probabilities of the words
        prob_this_class = math.log(prob_this_class)
        proba = 0
        for word in current_dict_test:
            proba += math.log(count_classes_dictn[class1][word]) - math.log(count_classes_dictn[class1]['total_count_class_words'])
        proba += prob_this_class
        if proba > max_proba:
            max_proba = proba
            result_class = class1
        if flag:
            flag = False
            max_proba = proba
            result_class = class1
    test_result.append(class1)
                

In [25]:
pickle_out = open('test_result.txt', 'wb')
pickle.dump(test_result, pickle_out)
pickle_out.close()

# Using gaussian naive bayes

Making a test data frame to be used in gaussian naive bayes (Can skip to run to save time)

In [ ]:
vec = np.arange(0, len(dictionary.keys()) * len(test_files)) * 0 + 1
test_df = pd.DataFrame(vec.reshape(-1, len(dictionary.keys())))
test_df.columns = dictionary.keys()
i = 0
for file in test_files:
    current_file = open(file[0], 'r')
    x = current_file.readlines()
    #go through each line
    for line in x:
        words = temp.tokenize(line)
        
        for word in punctuation:
            if word in words:
                words.remove(word)
        for word in stopWords:
            if word in words:
                words.remove(word)
        nw = copy.deepcopy(words)
        for word in nw:
            if word.isdigit():
                words.remove(word)
        
        #go through each word and build vocabulary with word count
        for singleWord in words:
            if(singleWord in dictionary):
                if(type(singleWord) is int):
                    print(singleWord)
                test_df[singleWord][i] += 1
    i += 1

Applying Gaussian Naive Bayes

In [24]:
clf = GaussianNB()
if 'result' in data_frame:
    del data_frame['result']
clf.fit(data_frame, train_y)
print(data_frame.shape)

D:\Vedant Mehra\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(19997, 2062)


In [25]:
if 'result' in test_data_frame:
    del test_data_frame['result']
y_pred_gaussian = clf.predict(test_data_frame)

Classification Report

In [26]:
print(classification_report(test_y, y_pred_gaussian))

                          precision    recall  f1-score   support

             alt.atheism       0.68      0.97      0.80       100
           comp.graphics       0.75      0.52      0.62       100
 comp.os.ms-windows.misc       0.72      0.52      0.60       100
comp.sys.ibm.pc.hardware       0.49      0.82      0.61       100
   comp.sys.mac.hardware       0.90      0.78      0.83       100
          comp.windows.x       0.90      0.72      0.80       100
            misc.forsale       0.62      0.95      0.75       100
               rec.autos       0.92      0.78      0.84       100
         rec.motorcycles       0.99      0.97      0.98       100
      rec.sport.baseball       0.99      1.00      1.00       100
        rec.sport.hockey       1.00      0.99      0.99       100
               sci.crypt       0.99      0.99      0.99       100
         sci.electronics       0.76      0.71      0.74       100
                 sci.med       0.99      0.94      0.96       100
         

In [ ]:
print(classification_report(test_y, test_result))